# SQL

Для анализа дан небольшой фаил размеров в 240 Мб содержащий только информацию о новостройках. Содержит срезы на каждый день(кроме дней поломок - простоя сервиса) Т.е. есть по каждому обекту есть информация за каждый день. Подучатеся в год(365 дней) информация об обект дублируется 365 раз.
Учтите что сложный селект может работать не быстро.

Для анализа использовать фаил с базой /home/synchronize_data/price_analytics/24.db.copied

In [5]:
!ls -la /home/synchronize_data/price_analytics/24.db.copied

-rw-r--r-- 1 synchronize_data synchronize_data 251686912 Jun  2 18:12 /home/synchronize_data/price_analytics/24.db.copied


# Пример использования: задача вывести самую дорогую квартиру из имеющихся.

Запустите следующие 6 строчек, они должны отработать корректно.

In [6]:
import matplotlib.pyplot as plt
import sqlite3
import json

In [7]:
connection = sqlite3.connect('/home/synchronize_data/price_analytics/24.db.copied')
cursor = connection.cursor()

Поля рекомендованные:
            id,
            "title",
            "price",
            "price_sale",
            date(datetime_create) as date,
            "brand",
            "apartment_area",
            "apartment_floor",
            "apartment_floors_total",
            "apartment_room",
            "apartment_ppm"

In [8]:
cursor.execute("""
SELECT 
    *
FROM copied
where 
    category = "Новостройки" 
    or category = "Реновация"
ORDER BY price desc
LIMIT 1
""")

In [9]:
flat = cursor.fetchall()

In [10]:
flat

[(44871689,
  'Саввинская 27 | 27 | 2 | 340.9 | Саввинская 27 | 55.730824,37.557959 | 1',
  847419076,
  847419076,
  '2024-06-02 06:13:37.130681',
  'Новостройки',
  'level.ru',
  'https://level.ru/old/sav27/apartment/6room/5-1/',
  340.9,
  2,
  6,
  6,
  2485829)]

In [136]:
# !pip install ipython-sql

In [137]:
# pip install --upgrade pip

In [11]:
%reload_ext sql 
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [12]:
%sql sqlite:////home/synchronize_data/price_analytics/24.db.copied
#%sql sqlite:///24.db.copied

In [162]:
%sql SELECT * from copied limit 5

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
598,"Причальный Bосток 29 65.3 Причальный пр., д. 8 55.76018,37.517482",37527037,26268925,2022-12-05 13:38:36.641439,Новостройки,level.ru,url_example,None,None,None,None,None
599,"Саввинская 17 17 4 82.5 55.734600731695195,37.565715889000494",202359478,202359478,2022-12-05 13:38:36.652957,Новостройки,level.ru,url_example,None,None,None,None,None
600,"Причальный Bосток 31 65.3 Причальный пр., д. 8 55.76018,37.517482",37802086,26461460,2022-12-05 13:38:36.662478,Новостройки,level.ru,url_example,None,None,None,None,None
601,"Саввинская 27 27 2 137.5 Саввинская 27 55.730669,37.557942",209072374,209072374,2022-12-05 13:38:36.670973,Новостройки,level.ru,url_example,None,None,None,None,None
602,"Причальный Юг 3 72.8 Причальный пр., д. 8 55.76018,37.517482",58793280,41155296,2022-12-05 13:38:36.724025,Новостройки,level.ru,url_example,None,None,None,None,None


# Задачи:

Выведите список новостроек в порядке убывания количества доступных к покупке квартир.

In [169]:
%%sql 
select
    title,
    count(*) as количество_квартир
from copied
where category = "Новостройки" and price > 0 and price is not NULL
group by title
order by количество_квартир desc
limit 5;

title,количество_квартир
1 1 1 -1 13.25 0,11180
14 1 1 -2 13.25 0,8045
10 1 1 -1 13.25 0,5082
14 1 1 -1 13.25 0,4707
10 1 1 2 13.25 0,4161


Найдите среднюю цену квадратного метра по всем новостройкам  

In [167]:
%%sql
select 
    avg(apartment_ppm) as средняя_цена_за_м2
from copied
where category = "Новостройки";

средняя_цена_за_м2
362162.48952185723


Выведите список новостроек в порядке убывания общего количества квартир.  

In [160]:
%%sql
select
    title,
    count(*) as количество_квартир
from copied
where category = "Новостройки"
group by title
order by количество_квартир desc
limit 5;

title,количество_квартир
1 1 1 -1 13.25 0,11180
14 1 1 -2 13.25 0,8045
10 1 1 -1 13.25 0,5082
14 1 1 -1 13.25 0,4707
10 1 1 2 13.25 0,4161


Найдите среднюю площадь квартир в новостройках с количеством комнат больше 3  

In [52]:
%%sql
select avg(apartment_area) as средняя_площадь
from copied
where category = "Новостройки" and apartment_room > 3;

средняя_площадь
120.00319930018476


Найдите среднюю цену квадратного метра по всем новостройкам на конкретный день.  

In [170]:
%%sql
select
    avg(apartment_ppm) AS средняя_цена
from copied
where category = 'Новостройки' and datetime_create LIKE '2023-12-05%';

средняя_цена
447008.8787564767


Найдите среднюю цен за квадратный метр в новостройках с количеством этажей больше 15.  

In [60]:
%%sql
select
    avg(apartment_ppm) as средняя_цена
from copied
where category = "Новостройки" and apartment_floors_total > 15;

средняя_цена
423575.7448402892


Найти все новостройки которые появились в 2023 году  

In [117]:
%%sql
select *
from copied
    where datetime_create like "2023%"
limit 5;

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
1932457,"Павелецкая Сити 9 2 23.8 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035",15336138,13035717,2023-01-01 00:23:50.275891,Новостройки,level.ru,https://level.ru/pav_city/flat/studio/9-5/,23.8,None,None,None,None
1932458,"Селигерская 1 3 17.8 пр-д Ильменский, вл. 12 55.859441,37.545155",8766862,6136803,2023-01-01 00:23:50.288006,Новостройки,level.ru,https://level.ru/selig/flat/1room/1-1-28/,17.8,None,None,None,None
1932459,"Стрешнево Север 2 45.1 Волоколамское ш., д. 81, корп. 2 55.82495356307498,37.411886931513436",15497134,10847993,2023-01-01 00:23:50.298460,Новостройки,level.ru,https://level.ru/stresh/apartment/2room/1-14/,45.1,None,None,None,None
1932460,"Южнопортовая 6 8 55.3 Г. Москва, ул. Южнопортовая, вл.28-28а 55.705727,37.696445",20175612,14122928,2023-01-01 00:23:50.309758,Новостройки,level.ru,https://level.ru/southport/flat/3room/6-75/,55.3,None,None,None,None
1932461,"Южнопортовая 1 39 78.3 Г. Москва, ул. Южнопортовая, вл.28-28а 55.705727,37.696445",26166406,18316484,2023-01-01 00:23:50.362469,Новостройки,level.ru,https://level.ru/southport/flat/4room/1-535/,78.3,None,None,None,None


Найти все новостройки которые имеют больше 50 квартир  

In [22]:
%%sql
select
    title, 
    count(*) as количество
from copied
where category = 'Новостройки'
group by title
having количество > 50
limit 5;

title,количество
1 1 1 -1 13.25 0,11180
1 1 1 10 29.16 S,74
1 1 1 11 40.01 S,90
1 1 1 13 112.04 E4,90
1 1 1 13 70.55 2,91


Найдите максимальную и минимальную цену среди всех новостроек  

In [128]:
%%sql
select *
from copied
where price = (select max(price) from copied)
limit 1;

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
44871689,"Саввинская 27 | 27 | 2 | 340.9 | Саввинская 27 | 55.730824,37.557959 | 1",847419076,847419076,2024-06-02 06:13:37.130681,Новостройки,level.ru,https://level.ru/old/sav27/apartment/6room/5-1/,340.9,2,6,6,2485829


In [14]:
%%sql
select *
from copied
where price_sale > 0
order by price_sale asc
limit 1;

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
31261045,10 1 1 1 3.8 1,551000,551000,2024-03-04 20:52:47.433973,Новостройки,gk-osnova.ru,https://api.gkosnova.tech/public/api/v1/building-objects/10/properties?detail=0,3.8,1,None,1,145000


Найти все новостройки в которых есть квартиры площадью больше 100 кв метров  

In [16]:
%%sql
select *
from copied
where apartment_area > 100
limit 5;

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
649,"Саввинская 27 27 2 137.5 Саввинская 27 55.730669,37.557942",209072374,209072374,2022-12-05 13:50:46.382455,Новостройки,level.ru,https://level.ru/sav27/apartment/3room/4-1/,137.5,None,None,None,None
651,"Саввинская 27 27 3 139.5 Саввинская 27 55.730669,37.557942",238985532,238985532,2022-12-05 13:50:46.400989,Новостройки,level.ru,https://level.ru/sav27/apartment/3room/4-4/,139.5,None,None,None,None
653,"Саввинская 17 17 4 110.5 55.734600731695195,37.565715889000494",269713058,269713058,2022-12-05 13:50:46.465808,Новостройки,level.ru,https://level.ru/sav17/flat/3room/2-9/,110.5,None,None,None,None
655,"Саввинская 27 27 6 189.9 Саввинская 27 55.730669,37.557942",564747927,564747927,2022-12-05 13:50:46.527201,Новостройки,level.ru,https://level.ru/sav27/apartment/3room/1-11/,189.9,None,None,None,None
662,"Мичуринский 6 15 112.3 Озерная ул., вл. 7 55.687726,37.471704",59713584,41799508,2022-12-05 13:53:27.017773,Новостройки,level.ru,https://level.ru/michur/flat/6room/6-2-51/,112.3,None,None,None,None


Найти новостройки котрые были построекны застройщщиком Донстрой  

In [17]:
%%sql
select *
from copied
where brand = "donstroy.moscow" and category = "Новостройки"
limit 5;

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
29629804,"Символ Независимость Корпус 19, 2 секц 4 27.1",10116430,10116430,2024-02-09 23:31:01.134710,Новостройки,donstroy.moscow,https://donstroy.moscow/objects/simvol/plans/quarter6/korpus19/section2/floor4/flat260119296/,27.1,4,24,100,None
29629805,"Символ Независимость Корпус 19, 2 секц 3 28.3",10125740,10125740,2024-02-09 23:31:01.143235,Новостройки,donstroy.moscow,https://donstroy.moscow/objects/simvol/plans/quarter6/korpus19/section2/floor3/flat260119283/,28.3,3,24,100,None
29629806,"Символ Независимость Корпус 19, 2 секц 4 27.1",10167920,10167920,2024-02-09 23:31:01.150161,Новостройки,donstroy.moscow,https://donstroy.moscow/objects/simvol/plans/quarter6/korpus19/section2/floor4/flat260119294/,27.1,4,24,100,None
29629807,"Символ Независимость Корпус 19, 2 секц 3 29.9",10620480,10620480,2024-02-09 23:31:01.158190,Новостройки,donstroy.moscow,https://donstroy.moscow/objects/simvol/plans/quarter6/korpus19/section2/floor3/flat260119274/,29.9,3,24,100,None
29629808,"Символ Независимость Корпус 19, 2 секц 5 27.1",10628620,10628620,2024-02-09 23:31:01.207331,Новостройки,donstroy.moscow,https://donstroy.moscow/objects/simvol/plans/quarter6/korpus19/section2/floor5/flat260119306/,27.1,5,24,100,None


In [ ]:
# для себя застройщтклв посмотреть

In [18]:
%%sql
select
    brand
from 
    copied
where 
    category = "Новостройки"
group by 
    brand

brand
Capital Group
donstroy.moscow
gk-osnova.ru
level.ru
Галс-Девелопмент


Найти всех застройщиков которые построили более 5 новостроек в 2023 году  

In [30]:
# %%sql
# select brand, title, datetime_create
# from copied
# where category = 'Новостройки' and datetime_create like "2023%"
# group by brand, title
# having count(title) > 5
# limit 5;

brand,title,datetime_create
Capital Group,"2 – комнатная квартира 107,00м2 49 этаж №306 в корпусе 3",2023-04-19 20:01:40.883713
Capital Group,"2 – комнатная квартира 107,00м2 50 этаж №313 в корпусе 3",2023-04-19 20:01:40.937124
Capital Group,"2 – комнатная квартира 107,00м2 51 этаж №320 в корпусе 3",2023-04-19 20:01:40.948741
Capital Group,"2 – комнатная квартира 107,00м2 52 этаж №327 в корпусе 3",2023-04-21 00:09:53.923230
Capital Group,"2 – комнатная квартира 107,00м2 54 этаж №341 в корпусе 3",2023-04-19 20:01:40.958880


Вывести список жилых комплексов с наибольшим количеством проданных квартир за последние 3 месяца. Вывести количество проданных квартир и процент от общего количества квартир этого комплекса

# Задачи с обогащением:

Построить месячный график количества продаж новостроек за последний год.  

Выведите список новостроек в западном районе.  

Найти все новостройки в ЗАО и СЗАО которые имеют больше 10 этажей  

Найти самый популярный размер квартир в каждом округе Москвы, и посчитать количеств продаж каждого размера ( сейчас за размер принимаем количество комнат)  

Найти все новостройки в пределах заданного радиуса от опеределнного места. И определить расстояние от каждой новостройки до заданного места, распределив их по возрастанию расстояния до заданного места.  

Построить график средней стоимости квадратного метра по месяцам, начиная с q2 2023 по q2 2024  

Найти все новостройки в которых есть квартиры с видом на определенный объект ( например парк, реку, останкинскую телебашню, москва сити или другую достопримечательность).

Показать информацию о количестве квартир с таким видом их средние площади и стоимости.  

Вывести список жилых комплексов с наибольшей разницей между стоимостью квартир в них и среденей стоимостью квартир в районе. Рассчитать процентное отклонение цены каждой квартиры от средней цены по району.  

Начиная с 2023 года, построить несколько графиков по месяцам : стоимости кв метра квартир, курса доллара(сколько рублей за один доллар), ключевой ставки, уровень безработиц, численность населения в городе, среднее количество детей в семье, количество сделок по ипотеке, количество сделок по льготным ипотечным программам,  

Найти все новостройки информация о которых появилась в 2023 году имеют рейтинг выше среднего по городу. Вывести список этих новостроек, среднее, минимальное и максимальное количество этажей в них, в разрезе типа квартир ( тип это количество комнат ) с количеством квартир.  

Вывести весь жк квартиры в которых были введены в эксплуатацию с чистовой отделкой в 2024 году  

вывести список жк бизнес класса в ВАО города Москвы района Измайлово, ключи от которых владельцы получили в 2023  

Выведите жк где соотношение количества квартир к количеств парковочных мест меньше 2.5  

Введите количество квартир в кирпичных домах города Москвы  

Определите среднее время оживания лита в домах застройщика level  

Выведите список жк от которых до ближайшей станции метро 10 минут пешком  

Выведите общее число объектов застройщиков которые больше 20 лет на рынке, и при качестве объектов выше среднего.

# Задачи на подумать и предложить

Выдвинуть 10 гипотез и проверить их на основе данных ( если данных не хватает то добавить данными из интернета)  

Гипотеза 1 - стоимость квартиры зависит от площади  

Гипотеза 2 - стоимость квартиры зависит от расстояния до центра города  

Гипотеза 3 -  